# XLM-RoBERTa-Spanish

In [1]:
from utils_v2 import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# Cargamos el dataset:
df = pd.read_csv('../train.csv',sep = '|')
df['Intencion'] = df.Intencion.str.findall(r'\d+').apply(lambda x: int(x[0]))

# Procesamos y dividimos las muestras en train y dev:
train_dataloader, validation_dataloader = tokenize_and_split(df, 
                                                             max_len=126, 
                                                             random_state=None, 
                                                             batch_size=8, 
                                                             test_size=0., 
                                                             transformer='xlm-roberta-spanish',
                                                             pad_token_id=0,
                                                             do_lower_case=True)

In [3]:
# Definimos el transformer que vamos a usar:
model, device = load_transformer(transformer='xlm-roberta-spanish',
                                 device='cuda:1',
                                 vocab_size=250002, 
                                 hidden_size=1024, 
                                 num_hidden_layers=24, 
                                 num_attention_heads=16, 
                                 intermediate_size=4096, 
                                 hidden_act='gelu', 
                                 hidden_dropout_prob=0.1, 
                                 attention_probs_dropout_prob=0.1, 
                                 max_position_embeddings=514, 
                                 type_vocab_size=1, 
                                 initializer_range=0.02, 
                                 layer_norm_eps=1e-12, 
                                 pad_token_id=1,
                                 bos_token_id=0, 
                                 eos_token_id=2,
                                 gradient_checkpointing=False, 
                                 num_labels=np.max(df['Intencion'].values)+1)

OSError: Can't load weights for 'xlm-roberta-large-finetuned-conll02-spanish'. Make sure that:

- 'xlm-roberta-large-finetuned-conll02-spanish' is a correct model identifier listed on 'https://huggingface.co/models'

- or 'xlm-roberta-large-finetuned-conll02-spanish' is the correct path to a directory containing a file named one of tf_model.h5, pytorch_model.bin.



In [4]:
# Entrenamiento
epochs = 10
lr = 1e-4
warmup_proportion = 0.1

num_training_steps =  len(train_dataloader) * epochs
num_warmup_steps = int(warmup_proportion * num_training_steps)
optimizer = AdamW(model.parameters(), weight_decay=0., lr=lr,correct_bias=False)
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=num_warmup_steps, 
                                            num_training_steps=num_training_steps)

train_loss_set = train(model,train_dataloader, validation_dataloader, optimizer, scheduler, epochs, device)

# plot training performance
plt.figure(figsize=(15,8))
plt.title("Training loss")
plt.xlabel("Batch")
plt.ylabel("Loss")
plt.plot(train_loss_set)
plt.show()

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 1; 10.92 GiB total capacity; 10.34 GiB already allocated; 9.50 MiB free; 10.39 GiB reserved in total by PyTorch)